In [ ]:
import geopandas as gpd
import pandas as pd
import pathlib
import requests
import configparser


In [ ]:

file_path = pathlib.Path(r"C:\Users\paulinkenbrandt\Documents\ArcGIS\Projects\MyProject2\Wellington_Openet_wgs84.geojson")

In [ ]:
station = "US-UTW"
config_path = f'../../station_config/{station}.ini'
config = configparser.ConfigParser()
config.read(config_path)

secrets_path = f'../../secrets/config.ini'
secrets_config = configparser.ConfigParser()
secrets_config.read(secrets_path)
url = secrets_config['DEFAULT']['url']
apikey = secrets_config['OPENET']['key']

In [ ]:


# 1. Read your GeoDataFrame (e.g., from a GeoJSON)
gdf = gpd.read_file(file_path)

# 2. Get the geometry of the first feature (Polygon)
polygon = gdf.geometry.iloc[0]

# 3. Extract the exterior coordinates as a list of (x, y) tuples
exterior_coords = list(polygon.exterior.coords)

# 4. Flatten that list of tuples into a single list
flattened_coords = [value for x, y in exterior_coords for value in (x, y)]


In [ ]:
flattened_coords

In [ ]:
import requests

# set your API key before making the request
header = {"Authorization": apikey}



In [ ]:
info_resp = requests.get(headers=header,
                         url="https://utah.openet-api.org/account/status").json()
print(info_resp)

In [ ]:
start_date = pd.to_datetime("2023-01-01")
end_date = pd.to_datetime("2024-10-01")
mon_req = info_resp['Monthly Requests'].split(" ")
used_req = int(mon_req[0])
limit_req = int(mon_req[-1])
date = start_date
while date < end_date and used_req < limit_req:
    bdate = date
    edate = date + pd.Timedelta('30d')

    # endpoint arguments
    args = {
      "date_range": [
        f"{bdate:%Y-%m-%d}",
        f"{edate:%Y-%m-%d}"
      ],
      "interval": "daily",
      "geometry": flattened_coords,
      "model": "Ensemble",
      "variable": "ET",
      "reference_et": "gridMET",
      "units": "mm",
      "encrypt": False
    }

    # query the api
    resp = requests.post(
        headers=header,
        json=args,
        url="https://utah.openet-api.org/raster/export/stack"
    )

    print(resp.json())


    if edate >= end_date:
        date = end_date
    else:
        date = edate

    used_req += 1
    print(date, used_req)